In [1]:
import gym
import torch
import numpy as np
import sys
from torchvision.transforms import ToTensor, Lambda
from torch import nn
from my_model import Network
from utils import dataset, dataloader, buffer
import trainer
import random
import torch.onnx as onnx
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('RLresult/cartpole/dqn')
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [2]:
env = gym.make('CartPole-v1') 
print("act", env.action_space)
print("obs", env.observation_space)

act Discrete(2)
obs Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [3]:
HIDDEN_SIZE = 32
BATCH_SIZE = 10000
learning_rate = 0.01
CART_ACTION = 2
CAPACITY = 10000
TRAIN_ITER = CAPACITY*10/BATCH_SIZE
PARAM_PATH = 'DQN_param/cartpoleDQN.pth'
PARAM_PATH_TEST = 'DQN_param/cartpoleDQN_test.pth'
#print(Transition(2,1,2,3))
#Transition(state=2, action=1, next_state=2, reward=3)

In [4]:
input_state_size = len(env.observation_space.sample())
updatedDQN = Network.DQN(input_state_size, HIDDEN_SIZE, CART_ACTION).to(device)
baseDQN = Network.DQN(input_state_size, HIDDEN_SIZE, CART_ACTION).to(device)

In [5]:
updatedDQN.load_state_dict(torch.load(PARAM_PATH_TEST))
baseDQN.load_state_dict(updatedDQN.state_dict())
baseDQN.eval()

DQN(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
    (5): ReLU()
  )
)

In [6]:
mybuffer = buffer.ManageMem(baseDQN, env, "cart")

In [7]:
mycartdata = dataset.EnvData(capacity = CAPACITY)

In [9]:
%%time

mydataloader = dataloader.CustomDataLoader(mycartdata, batch_size=BATCH_SIZE)
mytrain = trainer.Train("cart", mydataloader, updatedDQN, baseDQN)

CPU times: user 14 µs, sys: 2 µs, total: 16 µs
Wall time: 17.6 µs


In [10]:
optimizer = torch.optim.SGD(updatedDQN.parameters(), lr=learning_rate)
train_iter = 100
i = 0
while(i < train_iter):
    i = i + 1
    mybuffer.renewal_memory(CAPACITY, mycartdata)
    loss = mytrain.training(TRAIN_ITER, BATCH_SIZE, optimizer)
    writer.add_scalar("loss", loss, i)
    torch.save(updatedDQN.state_dict(), PARAM_PATH_TEST)
    baseDQN.load_state_dict(updatedDQN.state_dict())
    baseDQN.eval()
    
    
env.close()


loss =  tensor(0.2262, device='cuda:0', grad_fn=<MseLossBackward>)
loss =  tensor(0.2318, device='cuda:0', grad_fn=<MseLossBackward>)


KeyboardInterrupt: 

In [ ]:
writer.flush()
writer.close()